In [ ]:
# Llava Inference code from: https://huggingface.co/llava-hf/llava-1.5-13b-hf

In [ ]:
import requests
from PIL import Image

from google.colab import drive
import os

import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration

import json

model_id = "llava-hf/llava-1.5-13b-hf"


processor = AutoProcessor.from_pretrained(model_id)

model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to('cuda')




In [ ]:
conversation = [
    {

      "role": "user",
      "content": [
          {"type": "text", "text": "Please summarize the image in a detailed manner"},
          {"type": "image"},
        ],
    },
]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)


In [ ]:
drive.mount('/content/drive')

In [ ]:
files = os.listdir('/content/drive/MyDrive/final_dataset_images/final_dataset_images')

In [ ]:
summaries = []

for path in files:
  image = Image.open('/content/drive/MyDrive/final_dataset_images/final_dataset_images/' + path)
  inputs = processor(images=image, text=prompt, return_tensors='pt').to('cuda', torch.float16)
  output = model.generate(**inputs, max_new_tokens=100000, do_sample=False)
  begin_text_prompt_length = len("USER:   Please summarize the image in a detailed manner ASSISTANT: ")
  response = processor.decode(output[0], skip_special_tokens=True)[begin_text_prompt_length:]
  json_obj = {}
  json_obj['path'] =  path
  json_obj['summary'] = response
  summaries.append(json_obj)

In [ ]:
with open('image_summaries.json', 'w') as image_file:
  json.dump(summaries, image_file)